# Edward's Bowl of Spaghetti Code

## Imports and DataFrame Initialization

In [21]:
# TODO: KETE SETSUNASA NI WA
import pandas as pd
# TODO have input prompt instead of hard-coding
df_OLD = pd.read_excel("files/monthly_production_05-2019.xlsx")
df_NEW = pd.read_excel("files/mothpro2019.xlsx")

#dfRevTest = pd.read_excel('files/monthly_revenue_04-2019.xlsx').fillna(0)
dfCYPro = pd.read_excel('files/federal_production_CY08-17.xlsx')

## Reading & Setting Header Format

In [62]:
# Reads Header Config File
def read_hconfig():
    columns = []
    with open('config/headerdef.txt') as hdef:
        # Seperated by commas, no whitespace plz :(
        columns = hdef.readline().split(',')
    return columns

# Returns Header List based on Excel file
# Use if large amount of Field Names
def set_header(samplefile):
    return list(samplefile.columns)

def write_header(header):
    config = open("config/headerdef.txt","w")
    for field in header:
        config.write(field)
    config.close()

## Reading & Setting Unit Dictionary

In [1]:
# Reads Unit Config File
# Commodity and Unit seperated by an equals sign " = "
# Fast, Best for small number of Units / after initial read
def read_uconfig():
    units = {}
    with open('config/unitdef.txt') as udef:
        for line in udef:
            split = line.split(" = ")
            add_item(split[0], split[1].strip(), units)
    return units


# Returns Unit Dictionary on Excel file
# Very Slow, but better than manually setting large # of units
def make_unit_dict(file):
    units = {}
    for row in file['Product']:
        # Key and Value split
        line = split_unit(row)
        k,v = line[0], line[1]
        add_item(k,v,units)
    return units


# Returns Split String based on Commodity and Unit
def split_unit(string):
    # For general purpose commodities
    if "(" in string:
        split = string.rsplit(" (",1)
        split[1] = split[1].rstrip(")")
        return split
    # The comma is for Geothermal
    elif "," in string:
        return string.split(", ",1)
    # In case no unit is found
    return [string,'']



def add_item(key, value, dictionary):
    # Adds Value to Set if Key exists
    if key in dictionary:
        dictionary[key].add(value)
    # Else adds new key with value
    else:
        dictionary[key] = {value}

        
# Wrties to config to speed up process later
# Maybe have it write seperate files for each type?
def write_units(units):
    config = open("config/unitdef.txt","w")
    for k,v in units.items():
        for u in v:
            line = k + ' = ' +  u.strip("'")  + '\n'
            config.write(line)
    config.close()

In [6]:
fe18 = pd.read_excel('files/federal_production_CY03-18.xlsx')
fe18

,Calendar Year,Land Category,Land Class,State,County,FIPS Code,Offshore Region,Offshore Planning Area,Product,Volume
0,2003,Offshore,Federal,NaN,NaN,NaN,Offshore Alaska,Beaufort Sea,Gas (mcf),1.19997e+07
1,2003,Offshore,Federal,NaN,NaN,NaN,Offshore Alaska,Beaufort Sea,Oil (bbl),3.92576e+06
2,2003,Offshore,Federal,NaN,NaN,NaN,Offshore Gulf,Central Gulf of Mexico,Gas (mcf),3.4224e+09
3,2003,Offshore,Federal,NaN,NaN,NaN,Offshore Gulf,Central Gulf of Mexico,Oil (bbl),4.95406e+08
4,2003,Offshore,Federal,NaN,NaN,NaN,Offshore Gulf,Eastern Gulf of Mexico,Gas (mcf),28688973
5,2003,Offshore,Federal,NaN,NaN,NaN,Offshore Gulf,Eastern Gulf of Mexico,Oil (bbl),33654
6,2003,Offshore,Federal,NaN,NaN,NaN,Offshore Gulf,Western Gulf of Mexico,Gas (mcf),977513651
7,2003,Offshore,Federal,NaN,NaN,NaN,Offshore Gulf,Western Gulf of Mexico,Oil (bbl),65050709
8,2003,Offshore,Federal,NaN,NaN,NaN,Offshore Pacific,Southern California,Gas (mcf),55614925
9,2003,Offshore,Federal,NaN,NaN,NaN,Offshore Pacific,Southern California,Oil (bbl),27328228


## Check Header Function

In [63]:
def check_header(file):
    columns = file.columns
    uncheckedCols = set(columns)
    
    for i in range(len(header)):
        # Checks if Field in file and in correct column
        if columns.contains(header[i]):
            if columns[i] == header[i]:
                print(header[i] + ": True")
            else:
                print(header[i] + ": Wrong order")
            uncheckedCols.remove(header[i])
        else:
            # Field not present in the file
            print(header[i] + ': N/A')
    # Prints all fields not in the format
    print('\nNew Cols:', uncheckedCols)
        
header = set_header(dfCYPro)
check_header(fe18)

Calendar Year: True
Product: Wrong order
State/Offshore Region: N/A
FIPS Code: Wrong order
CPS/Planning Area: N/A
Production Volume: N/A
Onshore/Offshore: N/A
Category: N/A

New Cols: {'State', 'Offshore Region', 'Land Class', 'Land Category', 'County', 'Volume', 'Offshore Planning Area'}


## Unit Dictionary Check. Will have to do testing

In [69]:
fepro = pd.read_excel('files/federal_production_CY08-17.xlsx').fillna('')

def check_unit_dict(file, default):
    index = 0
    bad = False
    for u in file['Product']:
        # Splits line by Item and Unit
        line = split_unit(u)
        # Checks if Item is valid and has correct units
        if default.__contains__(line[0]):
            if line[1] not in default.get(line[0]):
                print('Row ' + str(index) + ': Unknown Unit - (' + line[1]  + ') [For Item: ' + line[0] + ']')
                bad = True
        else:
            print('Row ' + str(index) + ': Unknown Item: ' + line[0])
            bad = True
        index+=1
    if not bad:
        print('No Errors Found :)')
            

write_units(make_unit_dict(fe18))
default_dict = read_uconfig()
check_unit_dict(fepro ,default_dict)

Row 658: Unknown Item: Limestonse
Row 1423: Unknown Item: Limestonse
Row 2183: Unknown Item: Limestonse
Row 2937: Unknown Item: Limestonse
Row 3705: Unknown Item: Limestonse
Row 4470: Unknown Item: Limestonse
Row 5231: Unknown Item: Limestonse
Row 5996: Unknown Item: Limestonse


In [55]:
dfCYPro

,Calendar Year,Product,State/Offshore Region,FIPS Code,CPS/Planning Area,Production Volume,Onshore/Offshore,Category
0,2008,Gas (mcf),AK,02122,Kenai Peninsula Borough,3.4097e+07,Onshore,Federal
1,2008,Oil (bbl),AK,02122,Kenai Peninsula Borough,225208,Onshore,Federal
2,2008,Gas (mcf),AK,02185,North Slope Borough,5110,Onshore,Federal
3,2008,Oil (bbl),AK,02185,North Slope Borough,4321,Onshore,Federal
4,2008,Gas (mcf),AL,01003,Baldwin County,61346.9,Onshore,Federal
5,2008,Oil (bbl),AL,01003,Baldwin County,9.52366,Onshore,Federal
6,2008,Oil (bbl),AL,01039,Covington County,33575.8,Onshore,Federal
7,2008,Gas (mcf),AL,01053,Escambia County,2130.62,Onshore,Federal
8,2008,Oil (bbl),AL,01053,Escambia County,9771.25,Onshore,Federal
9,2008,Coal (tons),AL,01057,Fayette County,W,Onshore,Federal


In [58]:
w_count = 0
for v in dfCYPro['Production Volume']:
    if v == 'W':
        w_count += 1
w_count

451

In [38]:
print(len(dfCYPro.index))
dfCYPro['Production Volume'].replace('W',0).mean()
dfCYPro['Production Volume'].replace('W',0).std()

7560


117069856.77559274

In [1]:
import pickle
class Animal:
   def __init__(self, number_of_paws, color):
       self.number_of_paws = number_of_paws
       self.color = color
class Sheep(Animal):
   def __init__(self, color):
       Animal.__init__(self, 4, color)

mary = Sheep("white")
print (str.format("My sheep mary is {0} and has {1} paws", mary.color, mary.number_of_paws))
my_pickled_mary = pickle.dumps(mary)
print ("Would you like to see her pickled? Here she is!")
print (my_pickled_mary)

My sheep mary is white and has 4 paws
Would you like to see her pickled? Here she is!
b'\x80\x03c__main__\nSheep\nq\x00)\x81q\x01}q\x02(X\x0e\x00\x00\x00number_of_pawsq\x03K\x04X\x05\x00\x00\x00colorq\x04X\x05\x00\x00\x00whiteq\x05ub.'


In [6]:
import pickle
class Animal:
   def __init__(self, number_of_paws, color):
       self.number_of_paws = number_of_paws
       self.color = color
class Sheep(Animal):
   def __init__(self, color):
       Animal.__init__(self, 4, color)

mary = Sheep("white")
print (str.format("My sheep mary is {0} and has {1} paws", mary.color, mary.number_of_paws))
my_pickled_mary = pickle.dumps(mary)

with open('my_pickled_mary.bin', mode= 'wb') as binary_file:
    pickle.dump(mary, binary_file)
    
dolly = pickle.loads(my_pickled_mary)
print (str.format("My sheep dolly is {0} and has {1} paws", dolly.color, dolly.number_of_paws))


potato = [1,2,3,4,5,6]
with open('potato.bin', mode= 'wb') as file:
    pickle.dump(potato, file)
yam = pickle.load(open('potato.bin', 'rb'))
print(yam)

My sheep mary is white and has 4 paws
My sheep dolly is white and has 4 paws
[1, 2, 3, 4, 5, 6]
